In [1]:
from lib import MongoHandler, WeatherGetter
import pandas as pd
import sqlite3 
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())


In [2]:
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

Check the tables inside the database:

https://www.kaggle.com/laudanum/footballdelphi

In [3]:
print([i[0] for i in cur.execute("""SELECT * FROM sqlite_master;""").description])
cur.execute("""SELECT type, tbl_name FROM sqlite_master ORDER by type, tbl_name;""")
db = pd.DataFrame(cur.fetchall())
db.columns = [i[0] for i in cur.description]
db

['type', 'name', 'tbl_name', 'rootpage', 'sql']


,type,tbl_name
0,index,Matches
1,table,Matches
2,table,Teams
3,table,Teams_in_Matches
4,table,Unique_Teams
5,table,sqlite_sequence
6,view,FlatView
7,view,FlatView_Advanced
8,view,FlatView_Chrono_TeamOrder_Reduced


***Match_ID (int): unique ID per match***

Div (str): identifies the division the match was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League)

***Season (int): Season the match took place in (usually covering the period of August till May of the following year)***

Date (str): Date of the match

***HomeTeam (str): Name of the home team***

***AwayTeam (str): Name of the away team***

FTHG (int) (Full Time Home Goals): Number of goals scored by the home team

FTAG (int) (Full Time Away Goals): Number of goals scored by the away team

FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

In [4]:
cur.execute("""SELECT * FROM Matches LIMIT 1;""")
df_ml = pd.DataFrame(cur.fetchall())
df_ml.columns = [i[0] for i in cur.description]
df_ml

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H


***Season (str): Football season for which the data is valid***
    
***TeamName (str): Name of the team the data concerns***
    
KaderHome (str): Number of Players in the squad

AvgAgeHome (str): Average age of players

ForeignPlayersHome (str): Number of foreign players (non-German, non-English respectively) playing for the team

OverallMarketValueHome (str): Overall market value of the team pre-season in EUR (based on data from transfermarkt.de)

AvgMarketValueHome (str): Average market value (per player) of the team pre-season in EUR (based on data from transfermarkt.de)

StadiumCapacity (str): Maximum stadium capacity of the team's home stadium

In [5]:
cur.execute("""SELECT * FROM Teams LIMIT 1;""")
df_tl = pd.DataFrame(cur.fetchall())
df_tl.columns = [i[0] for i in cur.description]
df_tl

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000


***Match_ID (int): Unique match ID***
    
***Unique_Team_ID (int): Unique team ID (This table is used to easily retrieve each match a given team has played in)***

In [6]:
cur.execute("""SELECT * FROM Teams_in_Matches LIMIT 1;""")
df_tm = pd.DataFrame(cur.fetchall())
df_tm.columns = [i[0] for i in cur.description]
df_tm

,Match_ID,Unique_Team_ID
0,1,26


***TeamName (str): Name of a team***
    
***Unique_Team_ID (int): Unique identifier for each team***

In [7]:
cur.execute("""SELECT * FROM Unique_Teams LIMIT 1;""")
df_ut = pd.DataFrame(cur.fetchall())
df_ut.columns = [i[0] for i in cur.description]
df_ut

,TeamName,Unique_Team_ID
0,Bayern Munich,1


In [8]:
cur.execute("""SELECT * FROM FlatView LIMIT 1;""")
df_fv = pd.DataFrame(cur.fetchall())
df_fv.columns = [i[0] for i in cur.description]
df_fv

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AvgAgeHome,...,ForeignPlayersHome,ForeignplayersAways,KaderHome,KaderAway,StadiumCapacity,StadiumCapacityAway,AvgMarketValueHome,MarketValueAway,OverallMarketValueHome,OverallMarketValueAway
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H,25,...,8,13,29,32,47728,47728,305000,747000,8850000,23900000


In [9]:
cur.execute("""SELECT * FROM FlatView_Advanced LIMIT 1;""")
df_fva = pd.DataFrame(cur.fetchall())
df_fva.columns = [i[0] for i in cur.description]
df_fva

,Match_ID,Unique_Team_ID,Unique_Team,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,5772,1,Bayern Munich,D1,2005,2005-08-05,Bayern Munich,M'gladbach,3,0,H,25,27,14,171500000,6860000,75000


In [10]:
cur.execute("""SELECT * FROM FlatView_Chrono_TeamOrder_Reduced LIMIT 1;""")
df_fvc = pd.DataFrame(cur.fetchall())
df_fvc.columns = [i[0] for i in cur.description]
df_fvc

,Match_ID,Unique_Team_ID,TeamName,Unique_Team_ID:1,Match_ID:1,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Unique_Team
0,30071,1,Bayern Munich,1,30071,D1,1993,1993-08-07,Bayern Munich,Freiburg,3,1,H,Bayern Munich
